In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pip install -e ..

Obtaining file:///home/massimiliano/Projects/fl-g13
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for fl_g13 (pyproject.toml) ... done
  Created wheel for fl_g13: filename=fl_g13-0.0.1-py3-none-any.whl size=4783 sha256=8064e08b4f30ae57a5caf24381753035394944d203df3c3ab58bb159a922d71e
  Stored in directory: /tmp/pip-ephem-wheel-cache-y1bphrmu/wheels/ba/e6/76/85624c1c92362af95a88fb73ca068d24e3e74626054ee5ff32
Successfully built fl_g13
  Attempting uninstall: fl_g13
    Found existing installation: fl_g13 0.0.1
    Uninstalling fl_g13-0.0.1:
      Successfully uninstalled fl_g13-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [3]:
import wandb
import dotenv
import os

from pathlib import Path

import torch
import flwr
from flwr.simulation import run_simulation

from fl_g13.architectures import BaseDino
from fl_g13.fl_pytorch.client_app import get_client_app
from fl_g13.fl_pytorch.server_app import get_server_app

from fl_g13.fl_pytorch import build_fl_dependencies

dotenv.load_dotenv() # Load API Key
WANDB_API_KEY = os.getenv("WANDB_API_KEY")
wandb.login(key=WANDB_API_KEY)

print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")
build_fl_dependencies()

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Training on {device}")

# Specify the resources each of your clients need
# By default, each client will be allocated 1x CPU and 0x GPUs
backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

# When running on GPU, assign an entire GPU for each client
if device == "cuda":
    backend_config["client_resources"] = {"num_cpus": 1, "num_gpus": 1}
    # Refer to our Flower framework documentation for more details about Flower simulations
    # and how to set up the `backend_config`

2025-06-21 14:04:37.613 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: /home/massimiliano/Projects/fl-g13
/home/massimiliano/miniconda3/envs/fl-g13/lib/python3.11/site-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/massimiliano/.netrc
wandb: Currently logged in as: massimilianocarli (stefano-gamba-social-politecnico-di-torino) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Flower 1.17.0 / PyTorch 2.6.0+cu124
'vision_transformer.py' already exists.
'utils.py' already exists.
Training on cuda


## Hyperparameters

In [4]:
from fl_g13.modeling import load_or_create
from fl_g13.editing import SparseSGDM
from torch.optim import SGD

# Model Hyper-parameters
model_config={
    "head_layers": 3,
    "head_hidden_size": 512,
    "dropout_rate": 0.0,
    "unfreeze_blocks": 0,
}

# Training Hyper-parameters
batch_size = 64
lr = 1e-3
momentum = 0.9
weight_decay = 1e-5
T_max = 8 #! Unused
eta_min = 1e-5 #! Unused

# Model editing Hyper-parameters
model_editing = True
mask_type = 'local'
sparsity = 0.7
mask_calibration_round = 3
mask = None
model_editing_batch_size = 16

# Federated Hyper-parameters
K = 100
C = 0.1
J = 8
Nc = 50

save_every = 5
fraction_fit = C  # Sample of available clients for training
fraction_evaluate = 0.1  # Sample 50% of available clients for evaluation
min_fit_clients = 10  # Never sample less than 10 clients for training
min_evaluate_clients = 5  # Never sample less than 5 clients for evaluation
min_available_clients = 10  # Wait until all 10 clients are available

num_rounds = 200 + 100 # Add the current number of rounds
evaluate_each = 5
partition_type = 'shard'
NUM_CLIENTS = K

# Wandb config
use_wandb = True
project_name = "FL_client_model_editing_v4"

current_path = Path.cwd()
model_save_path = current_path / f"../models/fl_dino_v4/non_iid/fine-tuning"

## Training

In [5]:
print('-' * 200)
print(f"Training Non IId model")
print(f"Nc: {Nc}, J: {J}")
checkpoint_dir = f"{model_save_path}/{Nc}_{J}_{mask_type}_{mask_calibration_round}_{sparsity}"
print(f'checkpoint_dir:{checkpoint_dir}')

# Model
model, start_epoch = load_or_create(
    path=checkpoint_dir,
    model_class=BaseDino,
    model_config=model_config,
    optimizer=None,
    scheduler=None,
    device=device,
    verbose=True,
)
model.to(device)

optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)
criterion = torch.nn.CrossEntropyLoss()

## Unfreeze blocks
num_blocks = 12
model.unfreeze_blocks(num_blocks)
num_shards_per_partition = Nc

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params}")

os.makedirs(checkpoint_dir, exist_ok=True)
name = f"FL_Dino____model_non_iid_{Nc}_{J}_{mask_type}_{mask_calibration_round}_{sparsity}"

wandb_config = {
    # Wandb Params
    'name': name,
    'project_name': project_name,
    'run_id': f"{name}",
    # Federated Learning param
    "fraction_fit": fraction_fit,
    'partition_type': partition_type,
    'K': K,
    'C': C,
    'J': J,
    'Nc': Nc,
    # Model editing params
    'model_editing': model_editing,
    'mask_type': mask_type,
    'sparsity': sparsity,
    'model_editing_batch_size': model_editing_batch_size,
    'calibration_rounds': mask_calibration_round,
    # Training params
    'lr': lr,
    'momentum': momentum,
}

if model_editing:
    # Create a dummy mask for SparseSGDM
    init_mask = [torch.ones_like(p, device=p.device) for p in
                model.parameters()]  # Must be done AFTER the model is moved to the device
    # Optimizer
    optimizer = SparseSGDM(
        model.parameters(),
        mask=init_mask,
        lr=lr,
        momentum=0.9,
        weight_decay=1e-5
    )

client = get_client_app(
    model=model,
    optimizer=optimizer,
    criterion=criterion,
    device=device,
    partition_type=partition_type,
    local_epochs=1,
    local_steps=J,
    batch_size=batch_size,
    num_shards_per_partition=num_shards_per_partition,
    scheduler=None, #! Clients wont use scheduler, as it doesnt make sense here
    verbose=0,
    model_editing=model_editing,
    mask_type=mask_type,
    sparsity=sparsity,
    mask=mask,
    mask_calibration_round=mask_calibration_round,
    model_editing_batch_size=model_editing_batch_size,
    mask_func=None
)

compute_round = num_rounds + 1 - start_epoch
server = get_server_app(
    checkpoint_dir=checkpoint_dir,
    model_class=model,
    optimizer=optimizer,
    criterion=criterion,
    scheduler=None, #! Server wont use scheduler, as it doesnt make sense here
    num_rounds=compute_round,
    fraction_fit=fraction_fit,
    fraction_evaluate=fraction_evaluate,
    min_fit_clients=min_fit_clients,
    min_evaluate_clients=min_evaluate_clients,
    min_available_clients=min_available_clients,
    device=device,
    use_wandb=use_wandb,
    wandb_config=wandb_config,
    save_every=save_every,
    prefix='fl_baseline',
    evaluate_each=evaluate_each
)

# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config
)

wandb.finish()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Training Non IId model
Nc: 50, J: 8
checkpoint_dir:/home/massimiliano/Projects/fl-g13/notebooks/../models/fl_dino_v4/non_iid/fine-tuning/50_8_local_3_0.7
🔍 Loading checkpoint from /home/massimiliano/Projects/fl-g13/notebooks/../models/fl_dino_v4/non_iid/fine-tuning/50_8_local_3_0.7/fl_fl_baseline_BaseDino_epoch_200.pth
📦 Model class in checkpoint: BaseDino
🔧 Model configuration: {'variant': 'dino_vits16', 'dropout_rate': 0.0, 'head_hidden_size': 512, 'head_layers': 3, 'num_classes': 100, 'unfreeze_blocks': 0, 'activation_fn': 'GELU', 'pretrained': True}


Using cache found in /home/massimiliano/.cache/torch/hub/facebookresearch_dino_main
Using cache found in /home/massimiliano/.cache/torch/hub/facebookresearch_dino_main


➡️ Moved model to device: cuda
✅ Loaded checkpoint from /home/massimiliano/Projects/fl-g13/notebooks/../models/fl_dino_v4/non_iid/fine-tuning/50_8_local_3_0.7/fl_fl_baseline_BaseDino_epoch_200.pth, resuming at epoch 201
Trainable parameters: 22068068
🔍 Loading checkpoint from /home/massimiliano/Projects/fl-g13/notebooks/../models/fl_dino_v4/non_iid/fine-tuning/50_8_local_3_0.7/fl_fl_baseline_BaseDino_epoch_200.pth
📦 Model class in checkpoint: BaseDino
🔧 Model configuration: {'variant': 'dino_vits16', 'dropout_rate': 0.0, 'head_hidden_size': 512, 'head_layers': 3, 'num_classes': 100, 'unfreeze_blocks': 0, 'activation_fn': 'GELU', 'pretrained': True}


Using cache found in /home/massimiliano/.cache/torch/hub/facebookresearch_dino_main


➡️ Moved model to device: cuda
✅ Loaded checkpoint from /home/massimiliano/Projects/fl-g13/notebooks/../models/fl_dino_v4/non_iid/fine-tuning/50_8_local_3_0.7/fl_fl_baseline_BaseDino_epoch_200.pth, resuming at epoch 201
[Server] Server on device: cuda:0
[Server] CUDA available in client: True
Using strategy 'CustomFedAvg' (default option)


...

(ClientAppActor pid=13613) [Client] Client on device: cuda:0
(ClientAppActor pid=13613) [Client] CUDA available in client: True


Eval progress: 100%|██████████| 2/2 [00:00<00:00,  8.61batch/s]


(ClientAppActor pid=13613) [Client] Client on device: cuda:0
(ClientAppActor pid=13613) [Client] CUDA available in client: True


Eval progress: 100%|██████████| 2/2 [00:00<00:00,  8.54batch/s]
INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 100 round(s) in 6151.13s
INFO :      	History (loss, distributed):
INFO :      		round 1: 1.117174404859543
INFO :      		round 2: 1.1928792953491212
INFO :      		round 3: 1.0612184792757033
INFO :      		round 4: 1.0269935339689256
INFO :      		round 5: 0.9790092810988427
INFO :      		round 6: 1.0142654776573181
INFO :      		round 7: 0.8867003679275512
INFO :      		round 8: 0.8893357366323471
INFO :      		round 9: 0.9783381178975106
INFO :      		round 10: 0.9989013284444809
INFO :      		round 11: 0.8954063385725022
INFO :      		round 12: 0.9549557954072952
INFO :      		round 13: 0.8985259503126144
INFO :      		round 14: 0.945216891169548
INFO :      		round 15: 0.9087239801883698
INFO :      		round 16: 0.8447445183992386
INFO :      		round 17: 0.9136002153158188
INFO :      		round 18

centralized_accuracy,▁▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇██
centralized_eval_loss,█▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▂▁▁
decentralized_avg_eval_accuracy,▂▁▃▃▄▄▄▅▄▅▆▆▅▅▆▅▅▅▆▅▅▆▇▆▆▆▆▆▆▆▇▆▇█▇▇▇█▇▇
decentralized_avg_eval_loss,▇█▆▄▄▅▃▄▄▄▃▃▃▄▃▄▃▄▃▃▂▂▁▂▂▂▃▃▂▂▂▁▁▁▃▁▂▂▂▂
decentralized_avg_train_accuracy,▁▂▃▃▃▄▄▄▅▄▅▅▄▅▅▆▅▅▅▆▆▆▆▅▅▆▇▇▇▇▇▇▇▇██▇███
decentralized_avg_train_loss,██▇▆▆▆▆▅▄▄▄▅▄▄▄▄▃▄▄▃▃▄▃▃▃▂▂▃▂▂▂▂▂▂▂▂▁▂▂▁
centralized_accuracy,0.8008
centralized_eval_loss,0.66117
decentralized_avg_eval_accuracy,0.801
decentralized_avg_eval_loss,0.66502
decentralized_avg_train_accuracy,0.88513
